In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import autograd.numpy as np_   # Thinly-wrapped version of Numpy
from autograd import grad
import numpy as np

In [ ]:
df = pd.read_csv('dados_avc.csv')
df.head(2)


In [ ]:
# Preencher valores ausentes em 'bmi' com a média
df['bmi'].replace('N/A', float('nan'), inplace=True)
df['bmi'].fillna(df['bmi'].mean(), inplace=True)
# Criar faixas etárias
age_bins = [0, 18, 35, 60, 100]
age_labels = ['Criança/Adolescente', 'Jovem', 'Adulto', 'Idoso']
df['age'] = pd.cut(df['age'], bins=age_bins, labels=age_labels)

# Criar categorias de IMC
bmi_bins = [0, 18.5, 24.9, 29.9, 100]
bmi_labels = ['Abaixo do peso', 'Peso normal', 'Sobrepeso', 'Obesidade']
df['bmi'] = pd.cut(df['bmi'], bins=bmi_bins, labels=bmi_labels)

# Criar níveis de glicose no sangue
glucose_bins = [0, 70, 130, 200, 900]
glucose_labels = ['Baixo', 'Normal', "Alto", 'Muito alto']
df['avg_glucose_level'] = pd.cut(df['avg_glucose_level'], bins=glucose_bins, labels=glucose_labels)

# Converter colunas categóricas usando one-hot encoding
df = pd.get_dummies(df, columns=['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status','avg_glucose_level', 'bmi', 'age'], drop_first=True)
df.head(15)


In [ ]:
X = df.drop(['stroke', 'id'], axis=1)
Y = df['stroke']


In [ ]:
X.head(1)

In [ ]:
## converting the 0 to -1
X = X.replace(0, -1)

## convering the True to 1 and False to -1
Y = Y.replace(0, -1)
X = X.replace(True, 1)
X = X.replace(False, -1)

X = X.astype('float64')
Y = Y.astype('float64')

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [ ]:
def loss( parametros ):
    w, b, x, y = parametros
    est = w.T @ x + b
    mse = np_.mean( (est - y)**2)
    return mse

g = grad(loss)
x = np.array(X_train).T
y = np.array(Y_train)


In [ ]:
w = np.random.randn(x.shape[0], 1)
b = 0.0
alpha = 10**-2

for n in range(15000):
    grad_ = g( (w, b, x, y) )
    w -= alpha*grad_[0]
    b -= alpha*grad_[1]


In [ ]:
model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
Y_pred = np.round(Y_pred)

In [ ]:
def accuracy(y_test, y_est):
    return np.mean(np.sign(y_test)==np.sign(y_est))

## create tuples with value of w and X column name
w_tuples = []
w_lin_model = []
for i in range(len(w)):
    w_tuples.append((w[i][0], X.columns[i]))
    w_lin_model.append((model.coef_[i], X.columns[i]))

## sort the tuples
w_tuples.sort(reverse=True)
w_lin_model.sort(reverse=True)


In [ ]:
x = np.array(X_test).T
y = np.array(Y_test)

y_est = w.T @ x + b
print(f"Acurácia do modelo: {accuracy(y, y_est) * 100:.2f}%")

print("Acurácia da Biblioteca",accuracy_score(Y_test, Y_pred))

df_nosso_modelo = pd.DataFrame(w_tuples, columns=['Valor Nosso Modelo', 'Tipo'])
df_biblioteca = pd.DataFrame(w_lin_model, columns=['Valor Modelo Biblioteca', 'Tipo'])

df_nosso_modelo.head(8)

In [ ]:
df_biblioteca.head(8)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='entropy')

# Agora, vamos usar o método .fit() para ajustar os parâmetros da árvore:
tree.fit(X_train, Y_train)

# Podemos visualizar a árvore de decisão em uma figura!
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

plt.figure( figsize=(20,20) )
a = plot_tree(tree, feature_names=X.columns, fontsize=15, 
              node_ids=False, impurity=False, filled=True)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = tree.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print("Acurácia do modelo de árvore de decisão:", accuracy)


In [ ]:
import pandas as pd

# Obter importância das características
feature_importances = tree.feature_importances_

# Criar DataFrame com as features e sua importância
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Ordenar o DataFrame pela importância em ordem decrescente
feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)

# Exibir o DataFrame
print(feature_importance_df)
